In [1]:
cd ./drive/MyDrive

[Errno 2] No such file or directory: './drive/MyDrive'
/content


In [8]:
!pip install -q tensorflow==2.12.0
!pip install -q tensorflow-addons==0.20.0
!pip install -q git+https://github.com/hoyso48/tf-utils@main
!pip install --upgrade scipy
!pip install tf_utils

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 110.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.24.3 which is incompatible.
orbax-checkpoint 0.11.12 requires jax>=0.5.0, but you have jax 0.4.30 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.3 which is incompatible.
flax 0.10.6 requires jax>=0.5.1, but you have jax 0.4.30 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.3 which is incompatible.


  Using cached jax-0.5.1-py3-none-any.whl.metadata (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 109.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.30
    Uninstalling jaxlib-0.4.30:
      Successfully uninstalled jaxlib-0.4.30
  Attempting uninstall: jax
    Found existing installation: jax 0.4.30
    Uninstalling jax-0.4.30:
      Successfully uninstalled jax-0.4.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import matplotlib as mpl
import tensorflow.keras.mixed_precision as mixed_precision

from tqdm.autonotebook import tqdm
import sklearn

from tf_utils.schedules import OneCycleLR, ListedLR
from tf_utils.callbacks import Snapshot, SWA
from tf_utils.learners import FGM, AWP

import os
import time
import pickle
import math
import random
import sys
import cv2
import gc
import glob
import datetime

print(f'Tensorflow Version: {tf.__version__}')
print(f'Python Version: {sys.version}')

ModuleNotFoundError: No module named 'keras._tf_keras'

In [ ]:
# Seed all random number generators
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

def get_strategy(device='TPU'):
    if "TPU" in device:
        tpu = 'local' if device=='TPU-VM' else None
        print("connecting to TPU...")
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect(tpu=tpu)
        strategy = tf.distribute.TPUStrategy(tpu)
        IS_TPU = True

    if device == "GPU"  or device=="CPU":
        ngpu = len(tf.config.experimental.list_physical_devices('GPU'))
        if ngpu>1:
            print("Using multi GPU")
            strategy = tf.distribute.MirroredStrategy()
        elif ngpu==1:
            print("Using single GPU")
            strategy = tf.distribute.get_strategy()
        else:
            print("Using CPU")
            strategy = tf.distribute.get_strategy()
            CFG.device = "CPU"

    if device == "GPU":
        print("Num GPUs Available: ", ngpu)

    AUTO     = tf.data.experimental.AUTOTUNE
    REPLICAS = strategy.num_replicas_in_sync
    print(f'REPLICAS: {REPLICAS}')

    return strategy, REPLICAS, IS_TPU

STRATEGY, N_REPLICAS, IS_TPU = get_strategy()

In [ ]:
#NOTE: you should run KaggleDatasets.get_gcs_path(dataset_name) in the kaggle notebook to update gcs_path as they expires after several weeks..
#notebook: https://www.kaggle.com/code/hoyso48/islr-get-gcs-path

GCS_PATH = {
            'ISLR':'gs://kds-bf6468ebf8d3f83cb78ec34aab656e5f2249470042dabba5a5b022c4',
            '5fold':'gs://kds-3d2c4f3f70f22e7b8793b9d05e1e012047f4600d7ca765ccca45aa52',
            '5fold_randsplit':'gs://kds-cfd559924f5f32aae240670f7625264cb57d4c0152af5703f5e13a99',
            }

TRAIN_FILENAMES = tf.io.gfile.glob(GCS_PATH['5fold']+'/*.tfrecords')
COMPETITION_PATH = GCS_PATH['ISLR']

print(len(TRAIN_FILENAMES))
!gsutil cp {COMPETITION_PATH}/train.csv .
!gsutil cp {COMPETITION_PATH}/sign_to_prediction_index_map.json .

In [ ]:
# Train DataFrame
train_df = pd.read_csv('train.csv')
display(train_df.head())
display(train_df.info())

In [ ]:
import re
def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename.split('/')[-1]).group(1)) for filename in filenames]
    return np.sum(n)
print(count_data_items(TRAIN_FILENAMES), len(train_df))
assert count_data_items(TRAIN_FILENAMES) == len(train_df)

In [ ]:
ROWS_PER_FRAME = 543
MAX_LEN = 384
CROP_LEN = MAX_LEN
NUM_CLASSES  = 250
PAD = -100.
NOSE=[
    1,2,98,327
]
LNOSE = [98]
RNOSE = [327]
LIP = [ 0,
    61, 185, 40, 39, 37, 267, 269, 270, 409,
    291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
    78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
    95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
]
LLIP = [84,181,91,146,61,185,40,39,37,87,178,88,95,78,191,80,81,82]
RLIP = [314,405,321,375,291,409,270,269,267,317,402,318,324,308,415,310,311,312]

POSE = [500, 502, 504, 501, 503, 505, 512, 513]
LPOSE = [513,505,503,501]
RPOSE = [512,504,502,500]

REYE = [
    33, 7, 163, 144, 145, 153, 154, 155, 133,
    246, 161, 160, 159, 158, 157, 173,
]
LEYE = [
    263, 249, 390, 373, 374, 380, 381, 382, 362,
    466, 388, 387, 386, 385, 384, 398,
]

LHAND = np.arange(468, 489).tolist()
RHAND = np.arange(522, 543).tolist()

POINT_LANDMARKS = LIP + LHAND + RHAND + NOSE + REYE + LEYE #+POSE

NUM_NODES = len(POINT_LANDMARKS)
CHANNELS = 6*NUM_NODES

print(NUM_NODES)
print(CHANNELS)

def interp1d_(x, target_len, method='random'):
    length = tf.shape(x)[1]
    target_len = tf.maximum(1,target_len)
    if method == 'random':
        if tf.random.uniform(()) < 0.33:
            x = tf.image.resize(x, (target_len,tf.shape(x)[1]),'bilinear')
        else:
            if tf.random.uniform(()) < 0.5:
                x = tf.image.resize(x, (target_len,tf.shape(x)[1]),'bicubic')
            else:
                x = tf.image.resize(x, (target_len,tf.shape(x)[1]),'nearest')
    else:
        x = tf.image.resize(x, (target_len,tf.shape(x)[1]),method)
    return x

def tf_nan_mean(x, axis=0, keepdims=False):
    return tf.reduce_sum(tf.where(tf.math.is_nan(x), tf.zeros_like(x), x), axis=axis, keepdims=keepdims) / tf.reduce_sum(tf.where(tf.math.is_nan(x), tf.zeros_like(x), tf.ones_like(x)), axis=axis, keepdims=keepdims)

def tf_nan_std(x, center=None, axis=0, keepdims=False):
    if center is None:
        center = tf_nan_mean(x, axis=axis,  keepdims=True)
    d = x - center
    return tf.math.sqrt(tf_nan_mean(d * d, axis=axis, keepdims=keepdims))

class Preprocess(tf.keras.layers.Layer):
    def __init__(self, max_len=MAX_LEN, point_landmarks=POINT_LANDMARKS, **kwargs):
        super().__init__(**kwargs)
        self.max_len = max_len
        self.point_landmarks = point_landmarks

    def call(self, inputs):
        if tf.rank(inputs) == 3:
            x = inputs[None,...]
        else:
            x = inputs

        mean = tf_nan_mean(tf.gather(x, [17], axis=2), axis=[1,2], keepdims=True)
        mean = tf.where(tf.math.is_nan(mean), tf.constant(0.5,x.dtype), mean)
        x = tf.gather(x, self.point_landmarks, axis=2) #N,T,P,C
        std = tf_nan_std(x, center=mean, axis=[1,2], keepdims=True)

        x = (x - mean)/std

        if self.max_len is not None:
            x = x[:,:self.max_len]
        length = tf.shape(x)[1]
        x = x[...,:2]

        dx = tf.cond(tf.shape(x)[1]>1,lambda:tf.pad(x[:,1:] - x[:,:-1], [[0,0],[0,1],[0,0],[0,0]]),lambda:tf.zeros_like(x))

        dx2 = tf.cond(tf.shape(x)[1]>2,lambda:tf.pad(x[:,2:] - x[:,:-2], [[0,0],[0,2],[0,0],[0,0]]),lambda:tf.zeros_like(x))

        x = tf.concat([
            tf.reshape(x, (-1,length,2*len(self.point_landmarks))),
            tf.reshape(dx, (-1,length,2*len(self.point_landmarks))),
            tf.reshape(dx2, (-1,length,2*len(self.point_landmarks))),
        ], axis = -1)

        x = tf.where(tf.math.is_nan(x),tf.constant(0.,x.dtype),x)

        return x

In [ ]:
def decode_tfrec(record_bytes):
    features = tf.io.parse_single_example(record_bytes, {
        'coordinates': tf.io.FixedLenFeature([], tf.string),
        'sign': tf.io.FixedLenFeature([], tf.int64),
    })
    out = {}
    out['coordinates']  = tf.reshape(tf.io.decode_raw(features['coordinates'], tf.float32), (-1,ROWS_PER_FRAME,3))
    out['sign'] = features['sign']
    return out

def filter_nans_tf(x, ref_point=POINT_LANDMARKS):
    mask = tf.math.logical_not(tf.reduce_all(tf.math.is_nan(tf.gather(x,ref_point,axis=1)), axis=[-2,-1]))
    x = tf.boolean_mask(x, mask, axis=0)
    return x

def preprocess(x, augment=False, max_len=MAX_LEN):
    coord = x['coordinates']
    coord = filter_nans_tf(coord)
    if augment:
        coord = augment_fn(coord, max_len=max_len)
    coord = tf.ensure_shape(coord, (None,ROWS_PER_FRAME,3))

    return tf.cast(Preprocess(max_len=max_len)(coord)[0],tf.float32), tf.one_hot(x['sign'], NUM_CLASSES)

def flip_lr(x):
    x,y,z = tf.unstack(x, axis=-1)
    x = 1-x
    new_x = tf.stack([x,y,z], -1)
    new_x = tf.transpose(new_x, [1,0,2])
    lhand = tf.gather(new_x, LHAND, axis=0)
    rhand = tf.gather(new_x, RHAND, axis=0)
    new_x = tf.tensor_scatter_nd_update(new_x, tf.constant(LHAND)[...,None], rhand)
    new_x = tf.tensor_scatter_nd_update(new_x, tf.constant(RHAND)[...,None], lhand)
    llip = tf.gather(new_x, LLIP, axis=0)
    rlip = tf.gather(new_x, RLIP, axis=0)
    new_x = tf.tensor_scatter_nd_update(new_x, tf.constant(LLIP)[...,None], rlip)
    new_x = tf.tensor_scatter_nd_update(new_x, tf.constant(RLIP)[...,None], llip)
    lpose = tf.gather(new_x, LPOSE, axis=0)
    rpose = tf.gather(new_x, RPOSE, axis=0)
    new_x = tf.tensor_scatter_nd_update(new_x, tf.constant(LPOSE)[...,None], rpose)
    new_x = tf.tensor_scatter_nd_update(new_x, tf.constant(RPOSE)[...,None], lpose)
    leye = tf.gather(new_x, LEYE, axis=0)
    reye = tf.gather(new_x, REYE, axis=0)
    new_x = tf.tensor_scatter_nd_update(new_x, tf.constant(LEYE)[...,None], reye)
    new_x = tf.tensor_scatter_nd_update(new_x, tf.constant(REYE)[...,None], leye)
    lnose = tf.gather(new_x, LNOSE, axis=0)
    rnose = tf.gather(new_x, RNOSE, axis=0)
    new_x = tf.tensor_scatter_nd_update(new_x, tf.constant(LNOSE)[...,None], rnose)
    new_x = tf.tensor_scatter_nd_update(new_x, tf.constant(RNOSE)[...,None], lnose)
    new_x = tf.transpose(new_x, [1,0,2])
    return new_x

def resample(x, rate=(0.8,1.2)):
    rate = tf.random.uniform((), rate[0], rate[1])
    length = tf.shape(x)[0]
    new_size = tf.cast(rate*tf.cast(length,tf.float32), tf.int32) #(tf.cast(rate*tf.cast(length,tf.float32), tf.int32),tf.shape(x)[1])
    new_x = interp1d_(x, new_size) #tf.image.resize(x, new_size, method='bilinear')
    return new_x

def spatial_random_affine(xyz,
    scale  = (0.8,1.2),
    shear = (-0.15,0.15),
    shift  = (-0.1,0.1),
    degree = (-30,30),
):
    center = tf.constant([0.5,0.5])
    if scale is not None:
        scale = tf.random.uniform((),*scale)
        xyz = scale*xyz

    if shear is not None:
        xy = xyz[...,:2]
        z = xyz[...,2:]
        shear_x = shear_y = tf.random.uniform((),*shear)
        if tf.random.uniform(()) < 0.5:
            shear_x = 0.
        else:
            shear_y = 0.
        shear_mat = tf.identity([
            [1.,shear_x],
            [shear_y,1.]
        ])
        xy = xy @ shear_mat
        center = center + [shear_y, shear_x]
        xyz = tf.concat([xy,z], axis=-1)

    if degree is not None:
        xy = xyz[...,:2]
        z = xyz[...,2:]
        xy -= center
        degree = tf.random.uniform((),*degree)
        radian = degree/180*np.pi
        c = tf.math.cos(radian)
        s = tf.math.sin(radian)
        rotate_mat = tf.identity([
            [c,s],
            [-s, c],
        ])
        xy = xy @ rotate_mat
        xy = xy + center
        xyz = tf.concat([xy,z], axis=-1)

    if shift is not None:
        shift = tf.random.uniform((),*shift)
        xyz = xyz + shift

    return xyz

def temporal_crop(x, length=MAX_LEN):
    l = tf.shape(x)[0]
    offset = tf.random.uniform((), 0, tf.clip_by_value(l-length,1,length), dtype=tf.int32)
    x = x[offset:offset+length]
    return x

def temporal_mask(x, size=(0.2,0.4), mask_value=float('nan')):
    l = tf.shape(x)[0]
    mask_size = tf.random.uniform((), *size)
    mask_size = tf.cast(tf.cast(l, tf.float32) * mask_size, tf.int32)
    mask_offset = tf.random.uniform((), 0, tf.clip_by_value(l-mask_size,1,l), dtype=tf.int32)
    x = tf.tensor_scatter_nd_update(x,tf.range(mask_offset, mask_offset+mask_size)[...,None],tf.fill([mask_size,543,3],mask_value))
    return x

def spatial_mask(x, size=(0.2,0.4), mask_value=float('nan')):
    mask_offset_y = tf.random.uniform(())
    mask_offset_x = tf.random.uniform(())
    mask_size = tf.random.uniform((), *size)
    mask_x = (mask_offset_x<x[...,0]) & (x[...,0] < mask_offset_x + mask_size)
    mask_y = (mask_offset_y<x[...,1]) & (x[...,1] < mask_offset_y + mask_size)
    mask = mask_x & mask_y
    x = tf.where(mask[...,None], mask_value, x)
    return x

def augment_fn(x, always=False, max_len=None):
    if tf.random.uniform(())<0.8 or always:
        x = resample(x, (0.5,1.5))
    if tf.random.uniform(())<0.5 or always:
        x = flip_lr(x)
    if max_len is not None:
        x = temporal_crop(x, max_len)
    if tf.random.uniform(())<0.75 or always:
        x = spatial_random_affine(x)
    if tf.random.uniform(())<0.5 or always:
        x = temporal_mask(x)
    if tf.random.uniform(())<0.5 or always:
        x = spatial_mask(x)
    return x

def get_tfrec_dataset(tfrecords, batch_size=64, max_len=64, drop_remainder=False, augment=False, shuffle=False, repeat=False):
    # Initialize dataset with TFRecords
    ds = tf.data.TFRecordDataset(tfrecords, num_parallel_reads=tf.data.AUTOTUNE, compression_type='GZIP')
    ds = ds.map(decode_tfrec, tf.data.AUTOTUNE)
    ds = ds.map(lambda x: preprocess(x, augment=augment, max_len=max_len), tf.data.AUTOTUNE)

    if repeat:
        ds = ds.repeat()

    if shuffle:
        ds = ds.shuffle(shuffle)
        options = tf.data.Options()
        options.experimental_deterministic = (False)
        ds = ds.with_options(options)

    if batch_size:
        ds = ds.padded_batch(batch_size, padding_values=PAD, padded_shapes=([max_len,CHANNELS],[NUM_CLASSES]), drop_remainder=drop_remainder)

    ds = ds.prefetch(tf.data.AUTOTUNE)

    return ds


In [ ]:
def efficient_cutmix(samples, labels, alpha=0.7, cutmix_ratio=0.2):
    """
    高效的CutMix实现，使用向量化操作代替循环
    适用于手语识别的时序数据

    参数:
        samples: 输入特征 [batch_size, time_steps, channels]
        labels: 标签 [batch_size, num_classes]
        alpha: 混合权重
        cutmix_ratio: 要处理的批次中样本的比例
    """
    batch_size = tf.shape(samples)[0]

    # 仅在批次大小足够时执行CutMix
    def do_mix():
        # 创建随机排列的索引
        perm = tf.random.shuffle(tf.range(batch_size))

        # 创建CutMix掩码 - 决定哪些样本会被混合
        cutmix_samples = tf.cast(tf.random.uniform([batch_size]) < cutmix_ratio, tf.float32)
        cutmix_samples = tf.reshape(cutmix_samples, [-1, 1, 1])  # 广播到时间和特征维度

        # 创建手部和嘴唇区域的掩码
        # 预先计算这些掩码可以避免每个样本都执行复杂的索引操作
        channels = tf.shape(samples)[2]
        feature_mask = tf.zeros([1, 1, channels], dtype=tf.float32)

        # 构建简化的特征掩码 - 我们将样本分为三部分：
        # 1. 需要重点混合的区域 (掩码值 = 1.0)
        # 2. 轻度混合的区域 (掩码值 = 0.5)
        # 3. 保持原样的区域 (掩码值 = 0.0)

        # 随机生成一个三区域掩码，而不是精确针对手部/嘴唇
        # 这样可以大大提高效率，同时仍然提供多样化的混合
        random_mask = tf.random.uniform([1, 1, channels], 0, 1)

        # 创建三区域掩码
        high_mix_mask = tf.cast(random_mask < 0.3, tf.float32)  # 30% 通道重点混合
        medium_mix_mask = tf.cast((random_mask >= 0.3) & (random_mask < 0.6), tf.float32)  # 30% 通道轻度混合

        # 混合样本和置换样本
        mixed_samples = (
            (1 - cutmix_samples * high_mix_mask * 0.8) * samples +  # 原始样本，高混合区域减少80%
            (cutmix_samples * high_mix_mask * 0.8) * tf.gather(samples, perm) +  # 置换样本，高混合区域占80%

            (1 - cutmix_samples * medium_mix_mask * 0.4) * samples +  # 原始样本，中混合区域减少40%
            (cutmix_samples * medium_mix_mask * 0.4) * tf.gather(samples, perm)  # 置换样本，中混合区域占40%
        )

        # 混合标签 - 使用简单的线性插值
        cutmix_labels = tf.reshape(cutmix_samples, [-1, 1])  # 广播到标签维度
        mixed_labels = (
            (1 - cutmix_labels * alpha) * labels +
            (cutmix_labels * alpha) * tf.gather(labels, perm)
        )

        return mixed_samples, mixed_labels

    def no_mix():
        return samples, labels

    # 只有当批次大小 > 1 时才执行CutMix
    return tf.cond(batch_size > 1, do_mix, no_mix)

In [ ]:
from IPython.display import HTML
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation

def filter_nans(frames):
    return frames[~np.isnan(frames).all(axis=(-2,-1))]

ds = tf.data.TFRecordDataset(TRAIN_FILENAMES, num_parallel_reads=tf.data.AUTOTUNE, compression_type='GZIP')
ds = ds.map(decode_tfrec, tf.data.AUTOTUNE)
print(ds)
for x in ds:
    temp = x['coordinates'].numpy()
    if not len(filter_nans(temp[:,LHAND])) == 0:
        break

#调用数据增强
ds = get_tfrec_dataset(TRAIN_FILENAMES, augment=True, batch_size=1024)
# 打印数据集信息
for samples, labels in ds.take(1):
    print("Dataset samples shape:", samples.shape)
    print("Dataset labels shape:", labels.shape)

ds_cutmix = augment_dataset_with_cutmix(ds, cutmix_prob=0.5, cutmix_ratio=0.2)


edges = [(0,1),(1,2),(2,3),(3,4),(0,5),(0,17),(5,6),(6,7),(7,8),(5,9),(9,10),(10,11),(11,12),
         (9,13),(13,14),(14,15),(15,16),(13,17),(17,18),(18,19),(19,20)]

fig, ax = plt.subplots()

def plot_frame(frame, edges=[], idxs=[]):

    frame[np.isnan(frame)] = 0
    x = list(frame[...,0])
    y = list(frame[...,1])
    if len(idxs) == 0:
        idxs = list(range(len(x)))
    ax.clear()
    ax.scatter(x, y, color='dodgerblue')
    for i in range(len(x)):
        ax.text(x[i], y[i], idxs[i])

    for edge in edges:
        ax.plot([x[edge[0]], x[edge[1]]], [y[edge[0]], y[edge[1]]], color='salmon')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])

def animate_frames(frames, edges=[], idxs=[]):
    anim = FuncAnimation(fig, lambda frame: plot_frame(frame, edges, idxs), frames=frames, interval=100)
    return HTML(anim.to_jshtml())

In [ ]:
class ECA(tf.keras.layers.Layer):
    def __init__(self, kernel_size=5, **kwargs):
        super().__init__(**kwargs)
        self.supports_masking = True
        self.kernel_size = kernel_size
        self.conv = tf.keras.layers.Conv1D(1, kernel_size=kernel_size, strides=1, padding="same", use_bias=False)

    def call(self, inputs, mask=None):
        nn = tf.keras.layers.GlobalAveragePooling1D()(inputs, mask=mask)
        nn = tf.expand_dims(nn, -1)
        nn = self.conv(nn)
        nn = tf.squeeze(nn, -1)
        nn = tf.nn.sigmoid(nn)
        nn = nn[:,None,:]
        return inputs * nn

class LateDropout(tf.keras.layers.Layer):
    def __init__(self, rate, noise_shape=None, start_step=0, **kwargs):
        super().__init__(**kwargs)
        self.supports_masking = True
        self.rate = rate
        self.start_step = start_step
        self.dropout = tf.keras.layers.Dropout(rate, noise_shape=noise_shape)

    def build(self, input_shape):
        super().build(input_shape)
        agg = tf.VariableAggregation.ONLY_FIRST_REPLICA
        self._train_counter = tf.Variable(0, dtype="int64", aggregation=agg, trainable=False)

    def call(self, inputs, training=False):
        x = tf.cond(self._train_counter < self.start_step, lambda:inputs, lambda:self.dropout(inputs, training=training))
        if training:
          self._train_counter.assign_add(1)
        return x

class CausalDWConv1D(tf.keras.layers.Layer):
    def __init__(self,
        kernel_size=17,
        dilation_rate=1,
        use_bias=False,
        depthwise_initializer='glorot_uniform',
        name='', **kwargs):
        super().__init__(name=name,**kwargs)
        self.causal_pad = tf.keras.layers.ZeroPadding1D((dilation_rate*(kernel_size-1),0),name=name + '_pad')
        self.dw_conv = tf.keras.layers.DepthwiseConv1D(
                            kernel_size,
                            strides=1,
                            dilation_rate=dilation_rate,
                            padding='valid',
                            use_bias=use_bias,
                            depthwise_initializer=depthwise_initializer,
                            name=name + '_dwconv')
        self.supports_masking = True

    def call(self, inputs):
        x = self.causal_pad(inputs)
        x = self.dw_conv(x)
        return x

def Conv1DBlock(channel_size,
          kernel_size,
          dilation_rate=1,
          drop_rate=0.0,
          expand_ratio=2,
          se_ratio=0.25,
          activation='swish',
          name=None):
    '''
    efficient conv1d block, @hoyso48
    '''
    if name is None:
        name = str(tf.keras.backend.get_uid("mbblock"))
    # Expansion phase
    def apply(inputs):
        channels_in = tf.keras.backend.int_shape(inputs)[-1]
        channels_expand = channels_in * expand_ratio

        skip = inputs

        x = tf.keras.layers.Dense(
            channels_expand,
            use_bias=True,
            activation=activation,
            name=name + '_expand_conv')(inputs)

        # Depthwise Convolution
        x = CausalDWConv1D(kernel_size,
            dilation_rate=dilation_rate,
            use_bias=False,
            name=name + '_dwconv')(x)

        x = tf.keras.layers.BatchNormalization(momentum=0.95, name=name + '_bn')(x)

        x  = ECA()(x)

        x = tf.keras.layers.Dense(
            channel_size,
            use_bias=True,
            name=name + '_project_conv')(x)

        if drop_rate > 0:
            x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1), name=name + '_drop')(x)

        if (channels_in == channel_size):
            x = tf.keras.layers.add([x, skip], name=name + '_add')
        return x

    return apply

In [ ]:
class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, dim=256, num_heads=4, dropout=0, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim
        self.scale = self.dim ** -0.5
        self.num_heads = num_heads
        self.qkv = tf.keras.layers.Dense(3 * dim, use_bias=False)
        self.drop1 = tf.keras.layers.Dropout(dropout)
        self.proj = tf.keras.layers.Dense(dim, use_bias=False)
        self.supports_masking = True

    def call(self, inputs, mask=None):
        qkv = self.qkv(inputs)
        qkv = tf.keras.layers.Permute((2, 1, 3))(tf.keras.layers.Reshape((-1, self.num_heads, self.dim * 3 // self.num_heads))(qkv))
        q, k, v = tf.split(qkv, [self.dim // self.num_heads] * 3, axis=-1)

        attn = tf.matmul(q, k, transpose_b=True) * self.scale

        if mask is not None:
            mask = mask[:, None, None, :]

        attn = tf.keras.layers.Softmax(axis=-1)(attn, mask=mask)
        attn = self.drop1(attn)

        x = attn @ v
        x = tf.keras.layers.Reshape((-1, self.dim))(tf.keras.layers.Permute((2, 1, 3))(x))
        x = self.proj(x)
        return x

def TransformerBlock(dim=256, num_heads=4, expand=4, attn_dropout=0.2, drop_rate=0.2, activation='swish'):
    def apply(inputs):
        x = inputs
        x = tf.keras.layers.BatchNormalization(momentum=0.95)(x)
        x = MultiHeadSelfAttention(dim=dim,num_heads=num_heads,dropout=attn_dropout)(x)
        x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1))(x)
        x = tf.keras.layers.Add()([inputs, x])
        attn_out = x

        x = tf.keras.layers.BatchNormalization(momentum=0.95)(x)
        x = tf.keras.layers.Dense(dim*expand, use_bias=False, activation=activation)(x)
        x = tf.keras.layers.Dense(dim, use_bias=False)(x)
        x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1))(x)
        x = tf.keras.layers.Add()([attn_out, x])
        return x
    return apply

In [ ]:
# 新增
class WindowedAttention(tf.keras.layers.Layer):
    def __init__(self, dim=256, num_heads=4, window_size=16, shift=False, dropout=0.2):
        super().__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift = shift

        # 相对位置编码矩阵 (2*window_size-1)
        self.rel_pos_bias = self.add_weight(
            name="rel_pos_bias",
            shape=(2 * window_size - 1, num_heads),
            initializer="zeros",
        )

        self.qkv = tf.keras.layers.Dense(3*dim)
        self.proj = tf.keras.layers.Dense(dim)
        self.dropout = tf.keras.layers.Dropout(dropout)

    def get_relative_positions(self, length):
        # 生成相对位置索引矩阵
        range_vec = tf.range(length)
        distance_mat = range_vec[:, None] - range_vec[None, :]
        return distance_mat + self.window_size - 1  # 偏移至非负数

    def window_partition(self, x):
        # x: (B, T, C)
        B, T, C = tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[2]

        # 移位操作
        if self.shift:
            x = tf.roll(x, shift=-self.window_size//2, axis=1)

        # 填充与分窗
        pad_len = (self.window_size - T % self.window_size) % self.window_size
        x = tf.pad(x, [[0,0], [0,pad_len], [0,0]])
        x = tf.reshape(x, [B, -1, self.window_size, C])  # (B, num_win, win_size, C)
        return x, pad_len

    def call(self, x, mask=None):
        B, T, C = tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[2]

        # 分窗处理
        x_windows, pad_len = self.window_partition(x)  # (B, nW, win, C)
        x_windows = tf.reshape(x_windows, [-1, self.window_size, C])  # (B*nW, win, C)

        # 生成QKV
        qkv = self.qkv(x_windows)
        qkv = tf.reshape(qkv, [-1, self.window_size, 3, self.num_heads, C//self.num_heads])
        q, k, v = tf.unstack(qkv, axis=2)  # 各 (B*nW, win, nH, C/nH)

        # 相对位置编码
        rel_pos = self.get_relative_positions(self.window_size)
        rel_bias = tf.gather(self.rel_pos_bias, rel_pos)  # (win, win, nH)
        rel_bias = tf.transpose(rel_bias, [2,0,1])  # (nH, win, win)

        # 注意力计算
        attn = tf.einsum('bqhd,bkhd->bhqk', q, k)  # (B*nW, nH, win, win)
        attn = attn + rel_bias[None,...]  # 加入位置偏置
        attn = attn / tf.sqrt(tf.cast(C//self.num_heads, tf.float32))

        if mask is not None:
            # 继承原有mask机制
            mask = tf.reshape(mask, [B, -1, self.window_size])
            mask = tf.repeat(mask[:, None, :, :], self.num_heads, axis=1)
            attn = tf.where(mask, attn, -1e9)

        attn = tf.nn.softmax(attn, axis=-1)
        attn = self.dropout(attn)

        # 聚合Value
        out = tf.einsum('bhqk,bkhd->bqhd', attn, v)
        out = tf.reshape(out, [B, -1, self.dim])  # (B, T_pad, C)

        # 移除填充
        if pad_len > 0:
            out = out[:, :T, :]

        return self.proj(out)

In [ ]:
# 新增
def SwinTransformerBlock(dim=256, num_heads=4, window_size=16, shift=False, expand=4):
    def apply(inputs):
        x = inputs

        # 窗口注意力
        x_norm = tf.keras.layers.LayerNormalization()(x)
        attn = WindowedAttention(dim, num_heads, window_size, shift)(x_norm)
        x = x + attn  # 残差连接

        # 前馈网络
        x_norm = tf.keras.layers.LayerNormalization()(x)
        ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(dim * expand, activation='gelu'),
            tf.keras.layers.Dense(dim)
        ])(x_norm)
        x = x + ffn

        return x
    return apply

In [ ]:
# 新模型
def get_model(max_len=384, dim=192):
    inp = tf.keras.Input((max_len, CHANNELS))
    x = tf.keras.layers.Masking(mask_value=PAD)(inp)

    # Stem层（保留原卷积特征提取）
    x = tf.keras.layers.Dense(dim, use_bias=False, name='stem_conv')(x)
    x = tf.keras.layers.BatchNormalization(momentum=0.95, name='stem_bn')(x)

    # Stage 1: 小窗口细粒度特征
    x = Conv1DBlock(dim, 17, drop_rate=0.2)(x)
    x = SwinTransformerBlock(dim, window_size=32, shift=False)(x)
    x = Conv1DBlock(dim, 17, drop_rate=0.2)(x)
    x = SwinTransformerBlock(dim, window_size=32, shift=True)(x)  # 移位窗口

    # Stage 2: 中等窗口 + 下采样
    x = tf.keras.layers.AvgPool1D(2)(x)  # T=192
    x = Conv1DBlock(dim, 17, drop_rate=0.2)(x)
    x = SwinTransformerBlock(dim, window_size=24, shift=False)(x)
    x = Conv1DBlock(dim, 17, drop_rate=0.2)(x)
    x = SwinTransformerBlock(dim, window_size=24, shift=True)(x)

    # Stage 3: 大窗口全局上下文
    x = tf.keras.layers.AvgPool1D(2)(x)  # T=96
    x = Conv1DBlock(dim, 17, drop_rate=0.2)(x)
    x = SwinTransformerBlock(dim, window_size=48, shift=False)(x)
    x = Conv1DBlock(dim, 17, drop_rate=0.2)(x)
    x = SwinTransformerBlock(dim, window_size=48, shift=True)(x)

    # 分类头（保持原结构）
    x = tf.keras.layers.Dense(dim*2, activation=None, name='top_conv')(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = LateDropout(0.8, start_step=1000)(x)
    x = tf.keras.layers.Dense(NUM_CLASSES, name='classifier')(x)

    return tf.keras.Model(inp, x)

In [ ]:
# 首先定义一个自定义回调函数，用于根据当前epoch更新数据集
class DatasetUpdater(tf.keras.callbacks.Callback):
    def __init__(self, train_files, CFG, cutmix_epoch_start=50):
        super().__init__()
        self.train_files = train_files
        self.CFG = CFG
        self.cutmix_epoch_start = cutmix_epoch_start
        # 保存初始数据集以供第一个epoch使用
        self.initial_ds = get_tfrec_dataset(
            train_files,
            batch_size=CFG.batch_size,
            max_len=CFG.max_len,
            drop_remainder=True if 'all' != 'all' else False,
            augment=True,
            repeat=True,
            shuffle=32768
        )

    def on_epoch_begin(self, epoch, logs=None):
        # 注意：epoch从0开始计数
        if epoch + 1 >= self.cutmix_epoch_start:  # +1是为了按照人类习惯计数epoch
            # 创建带有CutMix的数据集
            ds = get_tfrec_dataset(
                self.train_files,
                batch_size=self.CFG.batch_size,
                max_len=self.CFG.max_len,
                drop_remainder=True if 'all' != 'all' else False,
                augment=True,
                repeat=True,
                shuffle=32768
            )
            ds = augment_dataset_with_cutmix(ds, cutmix_prob=0.2, cutmix_ratio=0.2, alpha=0.5)
            print(f"Epoch {epoch+1}: Using CutMix augmentation")
        else:
            # 使用没有CutMix的数据集
            ds = get_tfrec_dataset(
                self.train_files,
                batch_size=self.CFG.batch_size,
                max_len=self.CFG.max_len,
                drop_remainder=True if 'all' != 'all' else False,
                augment=True,
                repeat=True,
                shuffle=32768
            )
            print(f"Epoch {epoch+1}: Not using CutMix augmentation")

        # 更新模型的训练数据集
        self.model.train_dataset = ds

In [ ]:
def train_fold(CFG, fold, train_files, valid_files=None, strategy=STRATEGY, summary=True, cutmix_epoch_start=30):
    seed_everything(CFG.seed)
    tf.keras.backend.clear_session()
    gc.collect()
    tf.config.optimizer.set_jit(True)

    if CFG.fp16:
        try:
            policy = mixed_precision.Policy('mixed_bfloat16')
            mixed_precision.set_global_policy(policy)
        except:
            policy = mixed_precision.Policy('mixed_float16')
            mixed_precision.set_global_policy(policy)
    else:
        policy = mixed_precision.Policy('float32')
        mixed_precision.set_global_policy(policy)

    # 创建初始训练数据集（不带CutMix）
    if fold != 'all':
        train_ds = get_tfrec_dataset(
            train_files,
            batch_size=CFG.batch_size,
            max_len=CFG.max_len,
            drop_remainder=True,
            augment=True,
            repeat=True,
            shuffle=32768
        )

        valid_ds = get_tfrec_dataset(
            valid_files,
            batch_size=CFG.batch_size,
            max_len=CFG.max_len,
            drop_remainder=False,
            repeat=False,
            shuffle=False
        )
    else:
        train_ds = get_tfrec_dataset(
            train_files,
            batch_size=CFG.batch_size,
            max_len=CFG.max_len,
            drop_remainder=False,
            augment=True,
            repeat=True,
            shuffle=32768
        )
        valid_ds = None
        valid_files = []

    # 创建数据集更新器回调
    dataset_updater = DatasetUpdater(train_files, CFG, cutmix_epoch_start)

    num_train = count_data_items(train_files)
    num_valid = count_data_items(valid_files) if valid_files else 0
    steps_per_epoch = num_train//CFG.batch_size

    with strategy.scope():
        dropout_step = CFG.dropout_start_epoch * steps_per_epoch
        model = get_model(max_len=CFG.max_len, dropout_step=dropout_step, dim=CFG.dim)

        schedule = OneCycleLR(CFG.lr, CFG.epoch, warmup_epochs=CFG.epoch*CFG.warmup, steps_per_epoch=steps_per_epoch, resume_epoch=CFG.resume, decay_epochs=CFG.epoch, lr_min=CFG.lr_min, decay_type=CFG.decay_type, warmup_type='linear')
        decay_schedule = OneCycleLR(CFG.lr*CFG.weight_decay, CFG.epoch, warmup_epochs=CFG.epoch*CFG.warmup, steps_per_epoch=steps_per_epoch, resume_epoch=CFG.resume, decay_epochs=CFG.epoch, lr_min=CFG.lr_min*CFG.weight_decay, decay_type=CFG.decay_type, warmup_type='linear')

        awp_step = CFG.awp_start_epoch * steps_per_epoch
        if CFG.fgm:
            model = FGM(model.input, model.output, delta=CFG.awp_lambda, eps=0., start_step=awp_step)
        elif CFG.awp:
            model = AWP(model.input, model.output, delta=CFG.awp_lambda, eps=0., start_step=awp_step)

        opt = tfa.optimizers.RectifiedAdam(learning_rate=schedule, weight_decay=decay_schedule, sma_threshold=4, clipvalue=1.)
        opt = tfa.optimizers.Lookahead(opt,sync_period=5)

        model.compile(
            optimizer=opt,
            loss=[tf.keras.losses.CategoricalCrossentropy(from_logits=True)],
            metrics=[
                [
                tf.keras.metrics.CategoricalAccuracy(),
                ],
            ],
            steps_per_execution=steps_per_epoch,
        )

    if summary:
        print()
        model.summary()
        print()
        print(train_ds, valid_ds)
        print()
        schedule.plot()
        print()
        init=False
    print(f'---------fold{fold}---------')
    print(f'train:{num_train} valid:{num_valid}')
    print()

    if CFG.resume:
        print(f'resume from epoch{CFG.resume}')
        model.load_weights(f'{CFG.output_dir}/{CFG.comment}-fold{fold}-last.h5')
        if train_ds is not None:
            model.evaluate(train_ds.take(steps_per_epoch))
        if valid_ds is not None:
            model.evaluate(valid_ds)

    logger = tf.keras.callbacks.CSVLogger(f'{CFG.output_dir}/{CFG.comment}-fold{fold}-logs.csv')
    sv_loss = tf.keras.callbacks.ModelCheckpoint(f'{CFG.output_dir}/{CFG.comment}-fold{fold}-best.h5', monitor='val_loss', verbose=0, save_best_only=True,
                save_weights_only=True, mode='min', save_freq='epoch')
    snap = Snapshot(f'{CFG.output_dir}/{CFG.comment}-fold{fold}', CFG.snapshot_epochs)
    swa = SWA(f'{CFG.output_dir}/{CFG.comment}-fold{fold}', CFG.swa_epochs, strategy=strategy, train_ds=train_ds, valid_ds=valid_ds, valid_steps=-(num_valid//-CFG.batch_size))

    # 添加回调
    callbacks = [dataset_updater]  # 添加数据集更新器回调
    if CFG.save_output:
        callbacks.append(logger)
        callbacks.append(snap)
        callbacks.append(swa)
        if fold != 'all':
            callbacks.append(sv_loss)

    history = model.fit(
        train_ds,
        epochs=CFG.epoch-CFG.resume,
        steps_per_epoch=steps_per_epoch,
        callbacks=callbacks,
        validation_data=valid_ds,
        verbose=CFG.verbose,
        validation_steps=-(num_valid//-CFG.batch_size) if num_valid > 0 else None
    )

    if CFG.save_output:
        try:
            model.load_weights(f'{CFG.output_dir}/{CFG.comment}-fold{fold}-best.h5')
        except:
            pass
    if fold != 'all':
        cv = model.evaluate(valid_ds,verbose=CFG.verbose,steps=-(num_valid//-CFG.batch_size))
    else:
        cv = None

    return model, cv, history
def train_folds(CFG, folds, strategy=STRATEGY, summary=True):
    for fold in folds:
        if fold != 'all':
            all_files = TRAIN_FILENAMES
            train_files = [x for x in all_files if f'fold{fold}' not in x]
            valid_files = [x for x in all_files if f'fold{fold}' in x]
        else:
            train_files = TRAIN_FILENAMES
            valid_files = None

        train_fold(CFG, fold, train_files, valid_files, strategy=strategy, summary=summary)
    return

In [ ]:
class CFG:
    n_splits = 5
    save_output = True
    output_dir = '.'

    seed = 42
    verbose = 2 #0) silent 1) progress bar 2) one line per epoch

    max_len = 384
    replicas = 8
    lr = 5e-4 * replicas
    weight_decay = 0.1
    lr_min = 1e-6
    epoch = 400
    warmup = 0
    batch_size = 64 * replicas
    snapshot_epochs = []
    swa_epochs = [] #list(range(epoch//2,epoch+1))

    fp16 = True
    fgm = False
    awp = True
    awp_lambda = 0.2
    awp_start_epoch = 15
    dropout_start_epoch = 15
    resume = 0
    decay_type = 'cosine'
    dim = 192
    comment = f'islr-fp16-192-8-seed{seed}'

In [ ]:
# 定义训练和验证文件
all_files = TRAIN_FILENAMES
fold = 0  # 选择要训练的折
train_files = [x for x in all_files if f'fold{fold}' not in x]
valid_files = [x for x in all_files if f'fold{fold}' in x]

# 然后调用训练函数
train_fold(CFG, fold, train_files, valid_files, cutmix_epoch_start=30)

In [ ]:
# CFG.seed = 42
# CFG.comment = f'islr-fp16-192-8-seed{CFG.seed}'
# train_folds(CFG, ['all'], summary=False)

In [ ]:
# CFG.seed = 43
# CFG.comment = f'islr-fp16-192-8-seed{CFG.seed}'
# train_folds(CFG, ['all'], summary=False)

In [ ]:
# CFG.seed = 44
# CFG.comment = f'islr-fp16-192-8-seed{CFG.seed}'
# train_folds(CFG, ['all'], summary=False)

In [ ]:
# CFG.seed = 45
# CFG.comment = f'islr-fp16-192-8-seed{CFG.seed}'
# train_folds(CFG, ['all'], summary=False)